In [1]:
import random
import operator
import numpy as np
import pandas as pd

import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import operator

from deap import base
from deap import creator
from deap import gp
from deap import tools


pset = gp.PrimitiveSet("main", 9)
pset.addPrimitive(max, 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)
a=4
pset.addTerminal(4)


pset.renameArguments(ARG0="thickness")
pset.renameArguments(ARG1="size")
pset.renameArguments(ARG2="shape")
pset.renameArguments(ARG3="ma")
pset.renameArguments(ARG4="se")
pset.renameArguments(ARG5="bn")
pset.renameArguments(ARG6="bc")
pset.renameArguments(ARG7="nn")
pset.renameArguments(ARG8="mitoses")


expr = gp.genFull(pset, min_=1, max_=5)

creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=3, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.expr)



toolbox.register("row", tools.initRepeat, list, toolbox.individual, n=10)
toolbox.register("population", tools.initRepeat, list, toolbox.row, n=10)



x=toolbox.population()
print(x[1][1])
print(x[9][9])

tree = gp.PrimitiveTree(expr)
print(tree)



E:\Sofwares\Anaconda\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


mul(max(mul(size, add(bn, ma)), max(bn, mitoses)), mul(add(nn, ma), add(bn, bn)))
mul(mul(mul(mul(size, shape), max(shape, size)), mul(max(ma, nn), max(bn, thickness))), mul(mul(mul(bn, thickness), mul(bn, nn)), add(mul(shape, 4), max(nn, bc))))
max(shape, 4)


In [103]:
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer
from sklearn import preprocessing

def Make_Batches():
    data = pd.read_csv('Dataset_MLAI.txt',sep = ',',header = None)
    #display(data)
    data[6] =  pd.to_numeric(data[6], errors='coerce') 
    # drop nas
    data = data.dropna()

    x_features = data.iloc[:,1:10]
    y_features = data.iloc[:,10]
    y_list = list(y_features.values.T.ravel())
    x_array = x_features.values

    scaler = preprocessing.StandardScaler()
    scaled_x_l = []
    y_train_l = []
    for i in range(12):
        x_train, x_test, y_train, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state= random.randint(1,100),shuffle = True)
        scaled_x_l.append(scaler.fit_transform(x_train))
        y_train_l.append(y_train)
    return(scaled_x_l,y_train_l)
Make_Batches()

[[ 0.10263786  0.87447463  1.10163162  1.78572188  1.74435851  1.53950088
  -0.20791264  1.80788829  1.44870681]
 [-1.26586691 -0.80720735 -0.55081581 -0.66526893 -0.68963011 -0.49061017
   0.20791264 -0.48784287 -0.57274455]
 [-0.92374072 -0.80720735 -0.22032632 -0.31512739 -0.68963011 -0.74437405
  -0.62373792 -0.77480927 -0.57274455]
 [-1.26586691 -0.80720735 -0.8813053  -0.66526893 -0.68963011 -0.74437405
  -0.62373792 -0.77480927 -0.57274455]
 [-1.26586691 -0.80720735 -0.8813053  -0.66526893 -0.68963011  0.01691759
  -1.0395632  -0.77480927 -0.57274455]
 [ 0.78689024  1.54714743  1.10163162 -0.31512739  0.12169943  1.03197312
  -0.20791264  1.2339555   0.43798113]
 [-0.23948833 -0.80720735 -0.8813053  -0.31512739 -0.68963011 -0.74437405
  -1.0395632  -0.77480927 -0.57274455]
 [-1.26586691 -0.80720735 -0.8813053  -0.66526893 -0.68963011 -0.74437405
  -1.0395632  -0.77480927 -0.57274455]
 [-1.26586691 -0.80720735 -0.8813053  -0.66526893 -0.28396534 -0.74437405
  -1.0395632  -0.77480

([array([[-1.36880027, -1.03780325, -1.06391725, -0.78733599, -0.74869094,
          -1.03448446, -0.88285072, -0.78665681, -0.35977364],
         [ 1.15821562,  1.28069338,  1.33847653,  0.96229954,  1.04816731,
          -0.76225171,  0.03044313,  1.97175018,  1.3444173 ],
         [-0.10529233, -1.03780325, -1.06391725,  1.39970842, -0.29947637,
          -1.03448446, -0.88285072, -0.78665681, -0.35977364],
         [-0.7370463 , -1.03780325, -1.06391725, -0.34992711, -0.29947637,
          -0.2177862 , -1.33949764, -0.78665681, -0.35977364],
         [-1.05292329, -1.03780325, -1.06391725, -0.78733599, -0.74869094,
          -1.03448446, -1.33949764, -0.78665681, -0.35977364],
         [ 0.84233863,  1.94312098, -0.03431991,  0.52489066,  1.94659643,
           1.41561032,  1.85703082, -0.48016714, -0.35977364],
         [ 0.52646164, -0.04416184,  0.99527742,  0.52489066, -0.29947637,
           0.59891206,  1.4003839 ,  0.74579152, -0.35977364],
         [ 1.4740926 ,  1.28069338

In [8]:
# list42=[]
# tree_no=0
# fitnessM=[]
# for t1 in range(10):
#     for t2 in range(10):
#         tree_no=tree_no+1
#         print('tree : '+str(tree_no))
#         function = gp.compile(str(x[t1][t2]), pset)
#         for i in range(30):
#             if(function(scaled_X[i,0],scaled_X[i,1],scaled_X[i,2],scaled_X[i,3],scaled_X[i,4],scaled_X[i,5],scaled_X[i,6],scaled_X[i,7],scaled_X[i,8])<0):
#                 list42.append(4)
#             else:
#                 list42.append(2)
#         print("Batch 1 : ")
#         print(list42)
#         print("\n")

#         scaled_x_list = []
#         fitnessS=[]
#         for j in range(12):
#             tn=0
#             tp=0
#             fn=0
#             fp=0
#             x_train, x_test, y_train, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
#             listRand=[]
#             scaled_X = scaler.fit_transform(x_train)
#             scaled_x_list.append(scaled_X)
#             for i in range(30):
#                 if(function(scaled_X[i,0],scaled_X[i,1],scaled_X[i,2],scaled_X[i,3],scaled_X[i,4],scaled_X[i,5],scaled_X[i,6],scaled_X[i,7],scaled_X[i,8])<0):
#                     listRand.append(2)
#                 else:
#                     listRand.append(4)
#             print("Batch "+str(j+2)+" : ")

#             for n in range(30):
#                 if(listRand[n]==y_train[n]):
#                     if(y_train[n]==2):
#                         tn=tn+1
#                     else:
#                         tp=tp+1
#                 else:
#                     if(listRand[n]==2):
#                         fn=fn+1
#                     else:
#                         fp=fp+1



#             print(listRand)
#             print(y_train)
#             print("FP : "+str(fp)+" \n "+ "FN : "+str(fn)+" \n "+ "TP : "+str(tp)+" \n "+"TN : "+str(tn)+" \n ")
#             prec=tp/(tp+fp)
#             recall=tp/(tp+fn)
#             fitness=(2*prec*recall)/(prec+recall)
#             fitnessS.append(fitness)
#             print('Fitness = '+str(fitness*100))
#             print("\n")
#         fitnessM.append(fitnessS*100)
#         # print(x_train)
# print(scaled_x_list)

# print(fitnessM)


tree : 1
<function <lambda> at 0x00000249B983E488>
Batch 1 : 
[2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


Batch 2 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 2, 2, 2, 2, 4, 2, 4, 2, 2, 4, 4, 2, 4, 4, 2, 4, 4, 2, 2, 2, 2, 2, 4, 2, 2, 4, 4, 2]
FP : 19 
 FN : 0 
 TP : 11 
 TN : 0 
 
Fitness = 53.658536585365844


Batch 3 : 
[4, 4, 2, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 4, 2, 2, 4, 2, 4, 2, 2, 4, 2, 4, 4, 2, 4, 4, 2, 4, 2, 2, 2, 2, 2, 4, 2, 4, 2, 4, 4]
FP : 17 
 FN : 2 
 TP : 11 
 TN : 0 
 
Fitness = 53.658536585365844


Batch 4 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 4, 2, 4, 2, 2, 4, 2, 4, 4, 2, 4, 2, 4, 2, 2, 2, 4, 4, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 2]
FP : 17 
 FN : 2 
 TP : 11 
 TN : 0 
 
Fitness = 53.658536585365844


Batch 5 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4,

Batch 13 : 
[4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 4, 2, 2, 2, 4, 4, 2, 4, 4, 4, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 4, 4, 2, 2, 2, 2, 2]
FP : 18 
 FN : 2 
 TP : 10 
 TN : 0 
 
Fitness = 50.0


tree : 15
<function <lambda> at 0x00000249B983ED90>
Batch 1 : 
[2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 2, 4, 2, 2, 2, 4, 4, 4, 4, 4, 2, 4, 4, 4, 2, 4, 4, 4, 2, 4, 2, 4, 4, 4, 4, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 2, 4, 2, 2, 2, 4, 4, 4, 2, 2, 4, 2, 2, 2, 4, 2, 2, 4, 4, 2, 4, 4, 2, 4, 2, 2, 2, 2]
FP : 19 
 FN : 0 
 TP : 11 
 TN : 0 
 
Fitness = 53.658536585365844


Batch 10 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 2, 2, 2, 4, 2, 2, 2, 4, 2, 4, 2, 2, 2, 4, 4, 4, 4, 2, 2, 4, 2, 4, 2, 2, 4, 2, 2, 2, 2]
FP : 19 
 FN : 0 
 TP : 11 
 TN : 0 
 
Fitness = 53.658536585365844


Batch 11 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 2, 2, 2, 2, 4, 2, 4, 2, 2, 4, 4, 2, 4, 4, 2, 4, 4, 2, 2, 2, 2, 2, 4, 2, 2, 4, 4, 2]
FP : 19 
 FN : 0 
 TP : 11 
 TN : 0 
 
Fitness = 53.658536585365844


Batch 12 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 4, 2, 4, 2, 4, 4, 4, 2, 4, 2, 2, 4, 4, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
FP : 21 
 FN : 0 
 TP : 9 

Batch 12 : 
[2, 4, 2, 2, 4, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 2, 4, 4, 4, 4]
[4, 4, 2, 2, 2, 2, 4, 2, 4, 2, 4, 2, 4, 2, 4, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 2, 4, 4, 4, 4]
FP : 1 
 FN : 4 
 TP : 10 
 TN : 15 
 
Fitness = 80.0


Batch 13 : 
[2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 4, 4, 2, 2, 4]
[2, 2, 4, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 4, 4, 4, 2, 2, 4]
FP : 0 
 FN : 3 
 TP : 7 
 TN : 20 
 
Fitness = 82.35294117647058


tree : 36
<function <lambda> at 0x00000249B86E7048>
Batch 1 : 
[2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

Batch 8 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4]
[2, 4, 2, 2, 4, 4, 4, 4, 2, 4, 4, 4, 2, 2, 4, 2, 4, 4, 2, 4, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2]
FP : 15 
 FN : 0 
 TP : 14 
 TN : 1 
 
Fitness = 65.11627906976744


Batch 9 : 
[4, 4, 4, 4, 2, 2, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2]
FP : 20 
 FN : 0 
 TP : 6 
 TN : 4 
 
Fitness = 37.5


Batch 10 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 4, 2, 4, 2, 4, 4, 2, 2, 4, 4, 4, 4, 2, 4, 2, 4, 2, 2, 4, 2, 2, 4, 2, 4, 2, 2, 4, 2, 4]
FP : 14 
 FN : 0 
 TP : 16 
 TN : 0 
 
Fitness = 69.56521739130436


Batch 11 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 4, 2, 4, 2, 4, 4, 4, 2, 4, 2, 2, 4, 4, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
FP : 20 
 FN : 0 
 TP : 9 
 TN : 

[2, 2, 4, 2, 2, 2, 4, 2, 4, 4, 4, 2, 4, 2, 4, 2, 4, 4, 4, 4, 2, 4, 2, 2, 2, 4, 4, 4, 2, 2]
[2, 2, 4, 2, 2, 2, 4, 2, 2, 4, 2, 2, 4, 4, 4, 2, 2, 4, 2, 4, 2, 4, 2, 2, 2, 4, 4, 4, 4, 2]
FP : 4 
 FN : 2 
 TP : 11 
 TN : 13 
 
Fitness = 78.57142857142856


Batch 3 : 
[2, 2, 2, 4, 2, 2, 2, 4, 2, 4, 2, 2, 4, 2, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 2, 4, 2, 2, 2, 2]
[2, 2, 2, 4, 2, 2, 2, 4, 4, 4, 2, 2, 4, 2, 2, 2, 4, 2, 2, 4, 4, 2, 4, 4, 2, 4, 2, 2, 2, 2]
FP : 3 
 FN : 1 
 TP : 10 
 TN : 16 
 
Fitness = 83.33333333333333


Batch 4 : 
[4, 4, 2, 2, 4, 2, 2, 2, 2, 4, 4, 4, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 4, 2, 2, 4, 4, 4, 4]
[4, 4, 2, 2, 4, 2, 2, 2, 2, 4, 4, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 4, 2, 4, 4]
FP : 3 
 FN : 0 
 TP : 11 
 TN : 16 
 
Fitness = 88.0


Batch 5 : 
[4, 4, 2, 4, 4, 2, 2, 2, 2, 4, 4, 2, 2, 4, 4, 2, 4, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2]
[4, 4, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 4, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2]
FP : 4 
 FN : 1 
 TP : 7 
 TN : 18 
 
Fitness


Batch 2 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 4, 4, 2, 4, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2]
FP : 22 
 FN : 0 
 TP : 8 
 TN : 0 
 
Fitness = 42.10526315789474


Batch 3 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 4, 4, 4, 4, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 4, 2, 2, 4, 4, 2]
FP : 19 
 FN : 0 
 TP : 11 
 TN : 0 
 
Fitness = 53.658536585365844


Batch 4 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 4, 2, 2, 4, 2, 4, 2, 2, 4, 2, 4, 4, 2, 4, 4, 2, 4, 2, 2, 2, 2, 2, 4, 2, 4, 2, 4, 4]
FP : 17 
 FN : 0 
 TP : 13 
 TN : 0 
 
Fitness = 60.46511627906976


Batch 5 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 2, 2, 4, 2, 2, 4, 4, 2, 2, 2, 2, 2, 4, 4, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2]
FP : 22 
 FN : 0 
 T

Batch 7 : 
[2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 4, 2, 2, 4]
[4, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4]
FP : 3 
 FN : 4 
 TP : 4 
 TN : 19 
 
Fitness = 53.333333333333336


Batch 8 : 
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 4, 4, 2, 4, 2, 4, 4, 2, 4, 2, 4, 4, 4]
[4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 4, 2, 4, 4, 4, 4, 2, 4, 2, 2]
FP : 4 
 FN : 3 
 TP : 7 
 TN : 16 
 
Fitness = 66.66666666666666


Batch 9 : 
[4, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 4, 4, 2]
[4, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 4, 2, 4, 4, 2, 2, 2, 4, 2, 2, 4, 4, 2, 2, 2, 4, 4, 4, 2]
FP : 1 
 FN : 4 
 TP : 8 
 TN : 17 
 
Fitness = 76.1904761904762


Batch 10 : 
[4, 4, 2, 4, 2, 2, 2, 2, 4, 2, 2, 4, 2, 4, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2]
[4, 4, 2, 4, 2, 2, 4, 2, 4, 4, 2, 4, 2, 4, 2, 2, 2, 4, 4, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 2]
FP : 1 
 FN : 5 
 TP : 

Batch 6 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 2, 2, 4, 2, 2, 2, 2, 2, 4, 4, 2, 4, 2, 4, 2, 4, 2, 4, 4, 2, 2, 2, 2, 4, 4, 2, 2, 4, 2]
FP : 18 
 FN : 0 
 TP : 12 
 TN : 0 
 
Fitness = 57.14285714285715


Batch 7 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 4, 2, 4, 2, 2, 4, 2, 4, 4, 4, 2, 4, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4]
FP : 19 
 FN : 0 
 TP : 11 
 TN : 0 
 
Fitness = 53.658536585365844


Batch 8 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 4, 2, 2, 4, 4, 4, 4, 2, 4, 4, 4, 2, 2, 4, 2, 4, 4, 2, 4, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2]
FP : 16 
 FN : 0 
 TP : 14 
 TN : 0 
 
Fitness = 63.63636363636363


Batch 9 : 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 2, 2, 2, 2, 4, 4, 2, 4, 2, 4, 4, 4, 4, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4]
FP : 19 
 FN : 0 
 T

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [32]:
# scaler = preprocessing.StandardScaler()
# scaled_X_test = scaler.fit_transform(x_test)
# print(scaled_X_test)
# listRandtest=[]
# function = gp.compile(str(x[0][1]), pset)
# for i in range(len(y_test)-1):
#     if(function(scaled_X_test[i,0],scaled_X_test[i,1],scaled_X_test[i,2],scaled_X_test[i,3],scaled_X_test[i,4],scaled_X_test[i,5],scaled_X_test[i,6],scaled_X_test[i,7],scaled_X_test[i,8])<0):
#         listRandtest.append(2)
#     else:
#         listRandtest.append(4)
# print(listRandtest)
# print(y_test)

# tn1=0
# tp1=0
# fn1=0
# fp1=0


# for q in range(30):
#     if(listRandtest[q]==y_test[q]):
#         if(y_test[q]==2):
#             tn1=tn1+1
#         else:
#             tp1=tp1+1
#     else:
#         if(listRand[q]==2):
#             fn1=fn1+1
#         else:
#             fp1=fp1+1


           
# print("FP : "+str(fp1)+" \n "+ "FN : "+str(fn1)+" \n "+ "TP : "+str(tp1)+" \n "+"TN : "+str(tn1)+" \n ")
# prec1=tp1/(tp1+fp1)
# recall1=tp1/(tp1+fn1)
# fitness1=(2*prec1*recall1)/(prec1+recall1)
# print('Fitness = '+str(fitness1*100))


[[ 1.96754026  0.59636404  1.26728569 ...  1.85016852  2.01274466
  -0.35266901]
 [-0.8733747  -0.71043367 -0.74700451 ... -0.59549959 -0.61721485
  -0.35266901]
 [ 1.25731152  1.2497629   1.60300072 ...  1.44255717 -0.28846991
  -0.35266901]
 ...
 [ 1.61242589  0.59636404  0.59585562 ...  0.62733447 -0.61721485
  -0.35266901]
 [-1.22848907 -0.71043367 -0.74700451 ... -1.00311094 -0.61721485
  -0.35266901]
 [ 0.19196841 -0.71043367  0.26014059 ... -0.18788823 -0.28846991
  -0.35266901]]
[4, 4, 4, 4, 2, 4, 4, 4, 4, 2, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 4, 4, 2, 2, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 2, 4, 2, 4, 4, 2, 4, 2, 4, 4, 4, 4, 2, 4, 2, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 2, 4, 4, 2, 4, 4, 4, 2, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 2, 4, 4, 4, 4, 4, 2, 4, 4, 4, 2, 4, 2, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 2, 4, 4, 4, 4, 2, 4, 4, 4, 2, 4, 2, 4, 2, 4, 4, 4, 4, 4, 

In [117]:
def Pearson_evaluation(tree1 ,tree2):
    lsum_product = []; lsumx_product = [];  lsumy_product = [];
    scaled_x_lit,y_train_lit = Make_Batches()
    f_tree1 = Tree_fitness(tree1,scaled_x_lit,y_train_lit)
    f_tree2 = Tree_fitness(tree2,scaled_x_lit,y_train_lit)
    x_mean = np.mean(f_tree1)
    y_mean = np.mean(f_tree2)
    for j,k in zip(f_tree1,f_tree2):
        sub_x = j-x_mean; sub_y = k-y_mean
        product = (sub_x)*(sub_y)
        square_x = sub_x**2;square_y = sub_y**2

        lsum_product.append(product); lsumx_product.append(square_x); lsumy_product.append(square_y);
    x = np.sum(lsum_product)
    y = np.sqrt(np.sum(lsumx_product))
    z = np.sqrt(np.sum(lsumy_product))
    print(str(tree1))
    print(str(tree2))
    return( (x/(y*z))**2)
    
Pearson_evaluation(x[2][2],x[2][2])

[[-0.93541435 -0.56380401 -0.5634891  -0.69684919  3.16133749 -0.65654043
  -0.9009982  -0.48210508 -0.37907125]
 [ 0.40089186 -0.56380401 -0.5634891   0.98907626 -0.07710579 -0.65654043
  -0.50349899 -0.48210508 -0.37907125]
 [-0.26726124 -0.56380401 -0.5634891  -0.69684919 -0.53974055 -0.65654043
  -0.10599979 -0.48210508 -0.37907125]
 [-0.93541435 -0.56380401 -0.23839923 -0.3596641  -0.53974055 -0.65654043
  -0.10599979 -0.48210508 -0.37907125]
 [-0.26726124 -0.26706506 -0.23839923 -0.02247901 -0.53974055 -0.65654043
  -0.9009982  -0.48210508 -0.37907125]
 [-0.26726124 -0.56380401 -0.5634891  -0.69684919 -0.53974055 -0.65654043
  -0.50349899 -0.48210508 -0.37907125]
 [-0.93541435 -0.56380401 -0.5634891  -0.69684919 -0.53974055 -0.65654043
  -0.9009982  -0.48210508 -0.37907125]
 [ 2.07127462  0.6231518   0.73687036 -0.02247901  1.31079847  0.94044981
   1.48399703  2.34764213 -0.37907125]
 [-0.26726124 -0.56380401 -0.5634891  -0.02247901 -1.0023753  -0.65654043
  -0.10599979 -0.48210

1.0

In [31]:
import deap
x = toolbox.population()
for t1 in range(10):
    for t2 in range(9):
        ind1 = x[t1][t2]; ind2 = x[t1][t2+1]
        c=[]
        c=deap.gp.cxOnePoint(ind1,ind2)
        print(str(c[1]))
# for i in range(10):
#     for j in range(9):
#         child1, child2 = [toolbox.clone(ind) for ind in (x[i][j], x[i][j+1])]
#         tools.cxBlend(child1, child2, 0.5)
#         del child1.fitness.values
#         del child2.fitness.values
#         print(child1);print(child2)

add(mul(add(nn, thickness), add(mitoses, ma)), add(mul(thickness, se), max(shape, thickness)))
mul(max(mul(shape, add(mul(thickness, se), max(shape, thickness))), max(mul(bc, bc), shape)), max(mul(thickness, 4), mul(nn, bc)))
max(add(add(max(shape, se), mul(thickness, mitoses)), add(add(mitoses, thickness), max(thickness, shape))), add(mul(mul(thickness, bc), mul(shape, ma)), mul(mul(bc, 4), mul(mitoses, 4))))
mul(mul(max(mitoses, size), add(thickness, shape)), mul(mul(mitoses, max(size, size)), max(mul(shape, ma), thickness)))
mul(max(max(mul(max(ma, thickness), bc), bc), add(mitoses, se)), add(size, add(mul(size, add(nn, bn)), mul(add(size, nn), shape))))
max(add(mul(thickness, max(4, bc)), add(size, 4)), mul(ma, mul(shape, se)))
add(mul(max(ma, thickness), max(se, bc)), add(max(ma, mul(se, thickness)), mul(mitoses, se)))
add(max(max(add(bn, se), max(size, shape)), max(add(size, mitoses), add(shape, bc))), max(max(add(thickness, 4), mul(size, mitoses)), add(mul(4, nn), thickness)))
m

In [115]:
def Tree_fitness(tree,x,y):
    function = gp.compile(str(tree), pset)
    fitnessS=[]
    scaled_x_lit,y_train_lit = x,y
    for j in range(12):
        tn=0
        tp=0
        fn=0
        fp=0
        listRand=[]
        for i in range(30):
            if(function((scaled_x_lit[j])[i,0],(scaled_x_lit[j])[i,1],(scaled_x_lit[j])[i,2],(scaled_x_lit[j])[i,3],(scaled_x_lit[j])[i,4],(scaled_x_lit[j])[i,5],(scaled_x_lit[j])[i,6],(scaled_x_lit[j])[i,7],(scaled_x_lit[j])[i,8])<0):
                listRand.append(2)
            else:
                listRand.append(4)
#         print("Batch "+str(j+2)+" : ")

        for n in range(30):
            if(listRand[n]==(y_train_lit[j])[n]):
                if(y_train[n]==2):
                    tn=tn+1
                else:
                    tp=tp+1
            else:
                if(listRand[n]==2):
                    fn=fn+1
                else:
                    fp=fp+1
#         print("FP : "+str(fp)+" \n "+ "FN : "+str(fn)+" \n "+ "TP : "+str(tp)+" \n "+"TN : "+str(tn)+" \n ")
        prec=tp/(tp+fp)
        recall=tp/(tp+fn)
        fitness=(2*prec*recall)/(prec+recall)
        fitnessS.append(fitness*100)
    return(fitnessS)
# Tree_fitness(x[1][1])

In [111]:
Tree_fitness(x[1][1])

[[-1.26919198 -0.75058663 -0.7620735  -0.63755514 -0.56346659 -0.82813758
  -0.65016662 -0.66531205 -0.36089829]
 [ 1.2482136   1.50117325  1.87587322  0.68152446  1.04643796  1.55055548
   1.91628055  2.09829185  4.56044207]
 [-1.26919198 -0.75058663 -0.7620735  -0.63755514 -0.56346659 -0.82813758
  -1.16345605 -0.66531205 -0.36089829]
 [ 1.2482136   0.37529331  0.41034727  0.02198466 -0.56346659 -0.56383836
   0.88970168 -0.66531205 -0.36089829]
 [ 1.2482136   1.78264323  1.87587322 -0.63755514  2.65634251  1.02195702
  -0.13687718 -0.05117785 -0.36089829]
 [ 0.9335379   1.78264323  1.87587322 -0.63755514 -0.16099045  0.49335856
  -0.13687718  1.79122475 -0.36089829]
 [-0.01048919 -0.75058663 -0.7620735  -0.63755514 -0.56346659 -0.82813758
  -1.16345605 -0.66531205 -0.36089829]
 [-0.95451628 -0.75058663 -0.7620735  -0.63755514 -0.56346659 -0.82813758
  -1.16345605 -0.66531205 -0.36089829]
 [-1.26919198 -0.75058663 -0.7620735  -0.63755514 -0.56346659 -0.82813758
  -1.16345605 -0.66531

36.26638012525109